In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import altair as alt
import snowflake.snowpark as snowpark
from snowflake.snowpark.session import Session

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
use database Northwind;
use schema "dbo";

In [ ]:
df_orders = session.sql("""
    SELECT CAST("OrderDate" AS DATE) AS ORDERDATE, COUNT(*) AS ORDERCOUNT
    FROM "Orders"
    GROUP BY CAST("OrderDate" AS DATE)
    ORDER BY ORDERDATE
""").to_pandas()

st.altair_chart(
    alt.Chart(df_orders).mark_line(point=True).encode(
        x=alt.X("ORDERDATE:T", title="Date"),
        y=alt.Y("ORDERCOUNT:Q", title="Total Orders")
    ).properties(title="📦 Orders per Day", width=700),
    use_container_width=True
)

In [ ]:
df_top_products = session.sql("""
    SELECT TOP 5 "P"."ProductName", SUM("OD"."Quantity") AS "TotalSold"
    FROM "OrderDetails" "OD"
    JOIN "Products" "P" ON "OD"."ProductID" = "P"."ProductID"
    GROUP BY "P"."ProductName"
    ORDER BY "TotalSold" DESC
""").to_pandas()

st.altair_chart(
    alt.Chart(df_top_products).mark_bar().encode(
        x=alt.X("TotalSold:Q", title="Units Sold"),
        y=alt.Y("ProductName:N", sort='-x')
    ).properties(title="🏆 Top 5 Selling Products", width=700),
    use_container_width=True
)

In [ ]:
df_waffle = session.sql("""
    SELECT "O"."OrderDate", "OD"."Quantity"
    FROM "OrderDetails" "OD"
    JOIN "Orders" "O" ON "OD"."OrderID" = "O"."OrderID"
    WHERE "OD"."ProductID" = 78
    ORDER BY "O"."OrderDate"
""").to_pandas()

st.altair_chart(
    alt.Chart(df_waffle).mark_bar().encode(
        x=alt.X("OrderDate:T", title="Date"),
        y=alt.Y("Quantity:Q", title="Waffles Ordered")
    ).properties(title="🍫 Waffle Order Volume Over Time", width=700),
    use_container_width=True
)

In [ ]:
df_stock = session.sql("""
    SELECT "ProductName", "UnitsInStock"
    FROM "Products"
    WHERE "ProductID" = 78
""").to_pandas()

st.metric(label="🧊 Waffles In Stock", value=int(df_stock["UnitsInStock"].iloc[0]))

In [ ]:
df_live = session.sql("""
    SELECT TOP 10 
        "O"."OrderID", 
        "O"."OrderDate", 
        "C"."CompanyName", 
        "P"."ProductName", 
        "OD"."Quantity"
    FROM "OrderDetails" "OD"
    JOIN "Orders" "O" ON "OD"."OrderID" = "O"."OrderID"
    JOIN "Products" "P" ON "OD"."ProductID" = "P"."ProductID"
    JOIN "Customers" "C" ON "O"."CustomerID" = "C"."CustomerID"
    WHERE "O"."OrderDate" >= DATEADD(MINUTE, -5, CURRENT_TIMESTAMP)
    ORDER BY "O"."OrderDate" DESC
""").to_pandas()

st.dataframe(df_live, use_container_width=True)

st.write("🔄 To update, simply re-run this cell to fetch new live orders.")

In [ ]:
import pandas as pd
import streamlit as st
import altair as alt

# Query live data (last 5 minutes)
df_live_trend = session.sql("""
    SELECT 
        CAST("O"."OrderDate" AS DATETIME) AS "OrderTimestamp",
        SUM("OD"."Quantity") AS "Quantity"
    FROM "OrderDetails" "OD"
    JOIN "Orders" "O" ON "OD"."OrderID" = "O"."OrderID"
    WHERE "OD"."ProductID" = 78
      AND "O"."OrderDate" >= DATEADD(MINUTE, -5, CURRENT_TIMESTAMP)
    GROUP BY CAST("O"."OrderDate" AS DATETIME)
    ORDER BY "OrderTimestamp"
""").to_pandas()

# Calculate cumulative quantity
df_live_trend["CumulativeQuantity"] = df_live_trend["Quantity"].cumsum()

# Plot it
chart = alt.Chart(df_live_trend).mark_line(point=True).encode(
    x=alt.X("OrderTimestamp:T", title="Time"),
    y=alt.Y("CumulativeQuantity:Q", title="Total Units Ordered"),
    tooltip=["OrderTimestamp", "CumulativeQuantity"]
).properties(
    title="📈 Live Cumulative Waffle Orders (Last 5 Min)",
    width=800,
    height=400
)

st.altair_chart(chart, use_container_width=True)

st.metric(label="🧇 Waffles Ordered in Last 5 Min", value=int(df_live_trend["Quantity"].sum()))

st.write("🔁 Re-run this cell to update the live chart.")